In [232]:
import glob
import pickle
import pandas as pd
import networkx as nx
import multiprocessing as mp
import xml.etree.ElementTree as ET

* How large a group of co-authors does the average publication have?

Get all the articles and authors and count the authors.

* Do authors mostly publish using always the same group of authors?



* Do authors mainly reference papers with other authors with whom they've co-authored papers (including themselves)?


* What is the distribution in time for citations of papers in general, and for papers with the highest number of citations? Do they differ?
* Is there a correlation between citations and the number of keywords that papers share? I.e. papers which share the same subject cite each other more often.
* For the most-cited papers (define your own cutoff), is the correlation in shared keywords between them and the papers that cite them different from (5) 

In [236]:
def article_author(myroot):
    # Behold my xml tree parser.
    # If it works it aint stupid.

    full_name = []
    pubmed_id = []

    # First loop over every article in the xml file.
    for k in range(0, len(myroot)):
        # Then get the the article tag which is in the medlinecitation
        for child in myroot[k][0]:
            if child.tag == 'Article':
                # Then we need the author list. 
                for second_child in child:
                    if second_child.tag == 'AuthorList':
                        # Then in the authorlist we need all the authors for one paper.
                        for k2 in range(0, len(second_child)):
                            # For every author append the corresponding id to a list
                            pubmed_id.append(myroot[k][0][0].text)
                            # Getting the authors last name
                            lastname = second_child[k2][0].text
                            # Not all authors have their first name listed.
                            try:
                                firstname = second_child[k2][1].text
                            except:
                                firstname = '-'
                            # Create a full name and append it to a list
                            try:
                                name = str(firstname) + ' ' + str(lastname)
                            except:
                                name = 'None'

                            full_name.append(name)

    # Turn both lists created into a dataframe.                                                   
    db = pd.DataFrame((pubmed_id, full_name)).T

    return db

In [240]:
def databaser(files):
    print('working on: ', files)
    mytree = ET.parse(files)
    myroot = mytree.getroot()
    df = article_author(myroot)
    Full_df = Full_df.append(df)

    Full_df.to_pickle(f'/homes/rdalstra/Documents/Programming3/Assignment6/output/{files}.pkl')



In [241]:
def runner(files):
    cpus = 48
    with mp.Pool(cpus) as pool:
        results = pool.map(databaser, files)

In [ ]:
# if __name__ == "__main__":
#     Full_df = pd.DataFrame()
#     path = '/data/dataprocessing/NCBI/PubMed/'
#     files = glob.glob(path + '*.xml')
#     runner(files)

In [246]:
path = '/data/dataprocessing/NCBI/PubMed/'
files = glob.glob(path + '*.xml')

In [253]:
files[1][33:-4]

'pubmed21n0002'

In [254]:
check = pd.read_pickle('/homes/rdalstra/Documents/Programming3/Assignment6/output/test.pkl')

In [255]:
check

,0,1
0,22099534,Ashish Khandelwal
1,22099534,Ankur Gupta
2,22099534,Preeti Gupta
3,22099535,Maria Elisabetta Coccia
4,22099535,Francesca Rizzello
...,...,...
148816,22131297,Frances Shawyer
148817,22131295,Larissa V Stebounova
148818,22131295,Hallie Morgan
148819,22131295,Vicki H Grassian


In [245]:
check

,0,1
0,489576,D I Meyer
1,489576,M M Burger
2,489577,K P Mullinix
3,489577,M B Meyers
4,489577,J L Christmann
...,...,...
77996,519579,A Imlach
77997,519580,D Woods
77998,519582,D Woods
77999,519583,A Gilmore


In [ ]:
# testfile = 0717
'/data/dataprocessing/NCBI/PubMed/pubmed21n0717.xml'

In [222]:
# Question1.
total = len(Full_df) 

uniques = Full_df[0].nunique()

average_authors = round(total / uniques)

print('The papers have', average_authors, 'authors on average')

The papers have 5 authors on average


# TEST code

[
    [
        [
            [
                [
                    [

                        [
                            [
                                [
                                    [
                                        [
                                            [
                                                
                                            ]
                                        ]
                                    ]
                                ]
                    ]
                ]
            ]
        ]
    ]
]

In [151]:
for k in range(0, len(myroot)):
    for child in myroot[k][0]:
        try:
            print(child[0].text)
  

IndexError: child index out of range

In [58]:
names = []
pubmed_id = []
for title_name, firstname, lastname in zip(myroot.iter('PMID'), myroot.iter('ForeName'), myroot.iter('LastName')):
    full_name = [str(firstname.text) + ' ' + str(lastname.text)]
    names.append(full_name)
    pubmed_id.append(title_name.text)
article = pd.DataFrame((pubmed_id, names)).T

In [74]:
article

,0,1
0,30970,[P Tcherdakoff]
1,30971,[D Dubois]
2,30972,[J Petitcolas]
3,30973,[W Makowska]
4,30974,[E Zawisza]
...,...,...
29998,61530,[T L Mattes]
29999,61531,[L Z Klein]
30000,61532,[A Vidershaĭn]
30001,61533,[A H Beĭer]


In [120]:
pubmed_id = []
for title_name in myroot.iter('PMID'):
    pubmed_id.append(title_name.text)
article = pd.DataFrame(pubmed_id)

In [122]:
names = []
for firstname, lastname in zip(myroot.iter('ForeName'), myroot.iter('LastName')):
    full_name = [str(firstname.text) + ' ' + str(lastname.text)]
    names.append(full_name)
article_names = pd.DataFrame(names)

In [123]:
article_names

,0
0,Keinosuke Okazaki
1,Toru Watanabe
2,Isao Saito
3,Jun-Ichiro Murayama
4,Kohki Shinozaki
...,...
156244,Tomas L Takahata
156245,Tomas L Zhou
156246,Steven A Ye
156247,Steven A Sun
